In [1]:
import os

from xcube.core.store import new_data_store
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [2]:
# greate s3 data store
store_team = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    max_depth=10,
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [3]:
ERA5_AGGREGATION_ALL = [
    "d2m",
    "skt",
    "sp",
    "stl1",
    "stl2",
    "stl3",
    "stl4",
    "swvl1",
    "swvl2",
    "swvl3",
    "swvl4",
    "t2m",
    "u10",
    "v10",
    "vpd",
    "rh",
    "src",
    "fal",
]
ERA5_AGGREGATION_MEAN_MEDIAN_STD = ["lai_hv", "lai_lv"]
ERA5_AGGREGATION_SUM = [
    "e",
    "pev",
    "slhf",
    "sshf",
    "ssr",
    "ssrd",
    "str",
    "strd",
    "tp",
    "ro",
]
# ref Magnus Formula for vapor pressure (Gleichung 6)
# https://journals.ametsoc.org/view/journals/bams/86/2/bams-86-2-225.xml
MF_A = 17.625
MF_B = 243.04
MF_C = 610.94

In [4]:
data_id = "cubes/aux/era5_land_time_optimized.zarr"
era5 = store_team.open_data(data_id)
es = MF_C * np.exp((MF_A * era5["t2m"]) / (era5["t2m"] + MF_B))
e = MF_C * np.exp((MF_A * era5["d2m"]) / (era5["d2m"] + MF_B))
era5["rh"] = (e / es) * 100
era5["rh"].attrs = dict(
    standard_name="relative_humidity",
    long_name="Relative humidity (computed)",
    units="%",
)
era5["vpd"] = es - e
era5["vpd"].attrs = dict(
    standard_name="vapour_pressure_deficit",
    long_name="Vapour pressure deficit (computed)",
    units="Pa",
)
era5

<xarray.Dataset> Size: 859GB
Dimensions:  (time: 71592, lat: 250, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 64.95 64.85 64.75 64.65 ... 40.25 40.15 40.05
  * lon      (lon) float64 3kB -4.95 -4.85 -4.75 -4.65 ... 34.75 34.85 34.95
  * time     (time) datetime64[ns] 573kB 2016-11-01 ... 2024-12-31T23:00:00
Data variables: (12/30)
    d2m      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    e        (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    fal      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    lai_hv   (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    lai_lv   (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    pev      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    ...       ...
    t2m      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    tp       (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    u10      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    v10      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    rh       (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    vpd      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [5]:
%%time
era5_cube = era5.interp(
    lat=52,
    lon=10,
    method="linear",
)
era5_cube = era5_cube.drop(["lat", "lon"])
era5_cube

CPU times: user 5.47 s, sys: 132 ms, total: 5.6 s
Wall time: 5.6 s


<timed exec>:6: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.


<xarray.Dataset> Size: 9MB
Dimensions:  (time: 71592)
Coordinates:
  * time     (time) datetime64[ns] 573kB 2016-11-01 ... 2024-12-31T23:00:00
Data variables: (12/30)
    d2m      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    e        (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    fal      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    lai_hv   (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    lai_lv   (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    pev      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    ...       ...
    t2m      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    tp       (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    u10      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    v10      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    rh       (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
    vpd      (time) float32 286kB dask.array<chunksize=(1095,), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [6]:
def _aggregate_era5(era5: xr.Dataset, mode: str) -> xr.Dataset:
    era5_mode = getattr(era5.compute().resample(time="1D"), mode)()
    rename_dict = {key: f"era5_{key}_{mode}" for key in era5_mode}
    rename_dict["time"] = "time_era5"
    era5_mode = era5_mode.rename(rename_dict)
    for key in era5.data_vars:
        attrs = dict(
            standard_name=era5[key].attrs["standard_name"],
            long_name=era5[key].attrs["long_name"],
            units=era5[key].attrs["units"],
            institution="Copernicus Climate Data Store",
            source=(
                "https://cds.climate.copernicus.eu/datasets/"
                "reanalysis-era5-land?tab=overview"
            ),
            license=(
                "https://object-store.os-api.cci2.ecmwf.int/cci2-prod-catalogue/"
                "licences/licence-to-use-copernicus-products/licence-to-use-"
                "copernicus-products_b4b9451f54cffa16ecef5c912c9cebd6979925a95"
                "6e3fa677976e0cf198c2c18.pdf"
            ),
        )
        era5_mode[f"era5_{key}_{mode}"].attrs = attrs
    return era5_mode

In [7]:
%%time
era5_final = _aggregate_era5(era5_cube[ERA5_AGGREGATION_SUM], "sum")

e
{'GRIB_NV': 0, 'GRIB_Nx': 400, 'GRIB_Ny': 250, 'GRIB_cfName': 'lwe_thickness_of_water_evaporation_amount', 'GRIB_cfVarName': 'e', 'GRIB_dataType': 'fc', 'GRIB_gridDefinitionDescription': 'Latitude/Longitude Grid', 'GRIB_gridType': 'regular_ll', 'GRIB_iDirectionIncrementInDegrees': 0.1, 'GRIB_iScansNegatively': 0, 'GRIB_jDirectionIncrementInDegrees': 0.1, 'GRIB_jPointsAreConsecutive': 0, 'GRIB_jScansPositively': 0, 'GRIB_latitudeOfFirstGridPointInDegrees': 64.95, 'GRIB_latitudeOfLastGridPointInDegrees': 40.05, 'GRIB_longitudeOfFirstGridPointInDegrees': -4.95, 'GRIB_longitudeOfLastGridPointInDegrees': 34.95, 'GRIB_missingValue': 3.4028234663852886e+38, 'GRIB_name': 'Evaporation', 'GRIB_numberOfPoints': 100000, 'GRIB_paramId': 182, 'GRIB_shortName': 'e', 'GRIB_stepType': 'accum', 'GRIB_stepUnits': 1, 'GRIB_surface': 0.0, 'GRIB_totalNumber': 0, 'GRIB_typeOfLevel': 'surface', 'GRIB_units': 'm of water equivalent', 'GRIB_uvRelativeToGrid': 0, 'long_name': 'Evaporation', 'standard_name': 'l

In [8]:
%%time
era5_final = era5_final.update(
    _aggregate_era5(era5_cube[ERA5_AGGREGATION_ALL], "mean")
)

d2m
{'GRIB_NV': 0, 'GRIB_Nx': 400, 'GRIB_Ny': 250, 'GRIB_cfName': 'unknown', 'GRIB_cfVarName': 'd2m', 'GRIB_dataType': 'fc', 'GRIB_gridDefinitionDescription': 'Latitude/Longitude Grid', 'GRIB_gridType': 'regular_ll', 'GRIB_iDirectionIncrementInDegrees': 0.1, 'GRIB_iScansNegatively': 0, 'GRIB_jDirectionIncrementInDegrees': 0.1, 'GRIB_jPointsAreConsecutive': 0, 'GRIB_jScansPositively': 0, 'GRIB_latitudeOfFirstGridPointInDegrees': 64.95, 'GRIB_latitudeOfLastGridPointInDegrees': 40.05, 'GRIB_longitudeOfFirstGridPointInDegrees': -4.95, 'GRIB_longitudeOfLastGridPointInDegrees': 34.95, 'GRIB_missingValue': 3.4028234663852886e+38, 'GRIB_name': '2 metre dewpoint temperature', 'GRIB_numberOfPoints': 100000, 'GRIB_paramId': 168, 'GRIB_shortName': '2d', 'GRIB_stepType': 'instant', 'GRIB_stepUnits': 1, 'GRIB_surface': 0.0, 'GRIB_totalNumber': 0, 'GRIB_typeOfLevel': 'surface', 'GRIB_units': 'K', 'GRIB_uvRelativeToGrid': 0, 'long_name': '2 metre dewpoint temperature', 'standard_name': 'unknown', 'uni

In [9]:
%%time
era5_final = era5_final.update(
    _aggregate_era5(era5_cube[ERA5_AGGREGATION_MEAN_MEDIAN_STD], "mean")
)

lai_hv
{'GRIB_NV': 0, 'GRIB_Nx': 400, 'GRIB_Ny': 250, 'GRIB_cfName': 'unknown', 'GRIB_cfVarName': 'lai_hv', 'GRIB_dataType': 'fc', 'GRIB_gridDefinitionDescription': 'Latitude/Longitude Grid', 'GRIB_gridType': 'regular_ll', 'GRIB_iDirectionIncrementInDegrees': 0.1, 'GRIB_iScansNegatively': 0, 'GRIB_jDirectionIncrementInDegrees': 0.1, 'GRIB_jPointsAreConsecutive': 0, 'GRIB_jScansPositively': 0, 'GRIB_latitudeOfFirstGridPointInDegrees': 64.95, 'GRIB_latitudeOfLastGridPointInDegrees': 40.05, 'GRIB_longitudeOfFirstGridPointInDegrees': -4.95, 'GRIB_longitudeOfLastGridPointInDegrees': 34.95, 'GRIB_missingValue': 3.4028234663852886e+38, 'GRIB_name': 'Leaf area index, high vegetation', 'GRIB_numberOfPoints': 100000, 'GRIB_paramId': 67, 'GRIB_shortName': 'lai_hv', 'GRIB_stepType': 'instant', 'GRIB_stepUnits': 1, 'GRIB_surface': 0.0, 'GRIB_totalNumber': 0, 'GRIB_typeOfLevel': 'surface', 'GRIB_units': 'm**2 m**-2', 'GRIB_uvRelativeToGrid': 0, 'long_name': 'Leaf area index, high vegetation', 'stand

In [10]:
era5_final

<xarray.Dataset> Size: 740kB
Dimensions:           (time_era5: 2983)
Coordinates:
  * time_era5         (time_era5) datetime64[ns] 24kB 2016-11-01 ... 2024-12-31
Data variables: (12/30)
    era5_e_sum        (time_era5) float64 24kB -0.006948 -0.01207 ... -0.003727
    era5_pev_sum      (time_era5) float64 24kB -0.01444 -0.01618 ... -0.01392
    era5_slhf_sum     (time_era5) float64 24kB -1.738e+07 ... -9.348e+06
    era5_sshf_sum     (time_era5) float64 24kB 8.569e+05 1.119e+07 ... 8.827e+06
    era5_ssr_sum      (time_era5) float64 24kB 5.979e+07 5.584e+07 ... 3.06e+07
    era5_ssrd_sum     (time_era5) float64 24kB 6.916e+07 6.454e+07 ... 3.503e+07
    ...                ...
    era5_vpd_mean     (time_era5) float64 24kB 2.151e+05 1.796e+05 ... 2.775e+05
    era5_rh_mean      (time_era5) float64 24kB 97.34 97.63 97.44 ... 97.69 96.04
    era5_src_mean     (time_era5) float64 24kB 0.000134 0.0002963 ... 1.696e-05
    era5_fal_mean     (time_era5) float64 24kB 0.1354 0.1348 ... 0.1264 0.1262
    era5_lai_hv_mean  (time_era5) float64 24kB 1.968 1.966 1.965 ... 1.888 1.886
    era5_lai_lv_mean  (time_era5) float64 24kB 2.273 2.272 2.27 ... 2.173 2.171
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [12]:
for key in era5_final.data_vars:
    print(key)
    print(era5_final[key].attrs)

era5_e_sum
{'standard_name': 'lwe_thickness_of_water_evaporation_amount', 'long_name': 'Evaporation', 'units': 'm of water equivalent', 'institution': 'Copernicus Climate Data Store', 'source': 'https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview', 'license': 'https://object-store.os-api.cci2.ecmwf.int/cci2-prod-catalogue/licences/licence-to-use-copernicus-products/licence-to-use-copernicus-products_b4b9451f54cffa16ecef5c912c9cebd6979925a956e3fa677976e0cf198c2c18.pdf'}
era5_pev_sum
{'standard_name': 'unknown', 'long_name': 'Potential evaporation', 'units': 'm', 'institution': 'Copernicus Climate Data Store', 'source': 'https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview', 'license': 'https://object-store.os-api.cci2.ecmwf.int/cci2-prod-catalogue/licences/licence-to-use-copernicus-products/licence-to-use-copernicus-products_b4b9451f54cffa16ecef5c912c9cebd6979925a956e3fa677976e0cf198c2c18.pdf'}
era5_slhf_sum
{'standard_name': 'surface_upw

In [ ]:


    # aggregate from hourly to daily

    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_ALL], "min")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_ALL], "max")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_ALL], "median")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_ALL], "std")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_MEAN_MEDIAN_STD], "mean")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_MEAN_MEDIAN_STD], "median")
    )
    era5_final = era5_final.update(
        _aggregate_era5(era5_cube[ERA5_AGGREGATION_MEAN_MEDIAN_STD], "std")
    )
    era5_final = era5_final.chunk(chunks=dict(time_era5=era5_final.sizes["time_era5"]))
    era5_final = era5_final.astype(np.float32)